In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Section 1: Data Loading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb

# Load datasets
calendar = pd.read_csv('/kaggle/input/calender/calendar.csv')
sell_prices = pd.read_csv('/kaggle/input/sell-prices/sell_prices.csv')
sales_train_eval = pd.read_csv('/kaggle/input/sales-train-eval/sales_train_evaluation.csv')
sales_train_valid = pd.read_csv('/kaggle/input/sales-train-valid/sales_train_validation.csv')

## Section 2: Examining Datasets

## Load the Calendar Dataset

The `calendar.csv` file contains daily information such as dates, event names, and the mapping from `d_1, d_2, ...` to actual dates.

In [ ]:
print("Calendar shape:", calendar.shape)

# Display the first 5 rows
calendar.head()

In [ ]:
print("-- Missing values in calendar --")
print(calendar.isnull().sum())

print("\n-- Data types (info) --")
calendar.info()

print("\n-- Descriptive statistics --")
print(calendar.describe(include='all'))

## Load `sell_prices.csv`

This file contains information about the price of each item in each store, keyed by `wm_yr_wk`. We'll examine its shape and a few rows to understand how it aligns with the `calendar` and sales files.

In [ ]:
print("Sell Prices shape:", sell_prices.shape)

# Display first 5 rows
sell_prices.head()

In [ ]:
print("-- Missing values in sell_prices --")
print(sell_prices.isnull().sum())

print("\n-- Data types (info) --")
sell_prices.info()

print("\n-- Descriptive stats for sell_price --")
print(sell_prices["sell_price"].describe())

## Load the `sales_train_evaluation.csv`

This dataset holds the historical daily unit sales data per product and store from day 1 (d_1) to day 1941 (d_1941). It's a wide format with one column per day.

In [ ]:
print("Sales Evaluation shape:", sales_train_eval.shape)

# Display first 5 rows
sales_train_eval.head()

In [ ]:
print("-- Missing values in sales_eval --")
print(sales_train_eval.isnull().sum().sum())  # sum of all missing across columns

print("\n-- Data types (info) --")
sales_train_eval.info(verbose=False)  # 'verbose=False' to avoid printing all 1947 columns

### Additional Stats on `sales_train_evaluation`

Before melting or merging, let's see how many unique items, stores, departments, categories, and states we have in this dataset.

In [ ]:
print("Unique item IDs:", sales_train_eval["item_id"].nunique())
print("Unique store IDs:", sales_train_eval["store_id"].nunique())
print("Unique dept IDs:", sales_train_eval["dept_id"].nunique())
print("Unique cat IDs:", sales_train_eval["cat_id"].nunique())
print("Unique state IDs:", sales_train_eval["state_id"].nunique())

## Section 3: Store-by-Store Merge

To avoid running out of memory, we'll process each `store_id` separately:

1. Filter `sales_eval` for one store.
2. Melt the subset so `d_1 ... d_1941` become a "day" column and a "sales" column.
3. Merge with `calendar` on "d".
4. Filter `sell_prices` for that store and merge on `[store_id, item_id, wm_yr_wk]`.
5. Save the result as a separate `.pkl` file.
6. Clear variables and move on to the next store.

This way we never hold the full ~60 million rows in memory at once.

In [ ]:
import gc
import pandas as pd
import numpy as np

def reduce_memory_usage(df):
    """ Reduce memory usage of a dataframe by downcasting numerical types where possible. """
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype
        if col_type not in [object, 'category']:
            c_min, c_max = df[col].min(), df[col].max()
            if str(col_type).startswith('int'):
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                else:
                    df[col] = df[col].astype(np.int64)
            else:
                df[col] = df[col].astype(np.float32)
        else:
            if col_type == object and df[col].nunique() / len(df[col]) < 0.5:
                df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print(f"  Memory reduced: {start_mem:.2f} MB -> {end_mem:.2f} MB "
          f"(Reduced by {(start_mem - end_mem)/start_mem*100:.1f}%)")
    return df

def process_store(store, sales_eval, calendar, sell_prices, output_dir):
    """ Process data for a single store: filter, melt, merge, reduce memory, and save. """
    print(f"\nProcessing store: {store}")

    # Step 1: Filter sales data for the store
    df_store = sales_eval[sales_eval["store_id"] == store].copy()
    df_store = reduce_memory_usage(df_store)

    # Step 2: Melt the subset
    fixed_cols = ["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"]
    date_cols = [c for c in df_store.columns if c.startswith("d_")]
    df_melted_sub = pd.melt(df_store, id_vars=fixed_cols, value_vars=date_cols, var_name="d", value_name="sales")
    del df_store
    gc.collect()

    # Step 3: Merge with calendar
    df_cal_sub = pd.merge(df_melted_sub, calendar, how="left", on="d")
    del df_melted_sub
    gc.collect()

    # Step 4: Filter and merge sell prices
    sp_sub = sell_prices[sell_prices["store_id"] == store].copy()
    sp_sub = reduce_memory_usage(sp_sub)
    df_merged_sub = pd.merge(df_cal_sub, sp_sub, how="left", on=["store_id", "item_id", "wm_yr_wk"])
    del df_cal_sub, sp_sub
    gc.collect()

    # Step 5: Reduce memory usage again
    df_merged_sub = reduce_memory_usage(df_merged_sub)

    # Step 6: Save to disk
    out_path = f"{output_dir}/merged_{store}.pkl"
    df_merged_sub.to_pickle(out_path)
    print(f"  Saved merged data for store={store}, shape={df_merged_sub.shape} -> {out_path}")

    # Step 7: Clear memory
    del df_merged_sub
    gc.collect()

# Make sure dataframes are loaded before calling this
output_directory = "/kaggle/working"

all_stores = sales_train_eval["store_id"].unique()
print("Found store_ids:", all_stores)

for store in all_stores:
    process_store(store, sales_train_eval, calendar, sell_prices, output_directory)

## Section 4: Baseline Model (Naive)

1. **Read** the wide `sales_train_evaluation.csv`, which has 30,490 rows (`_evaluation` IDs).  
2. **Compute** the average sales for the last 28 days (`d_1914 ~ d_1941`) for each row, calling that `naive_mean`.  
3. **Replicate** those rows as `_validation` by replacing `_evaluation` with `_validation` in the `id` string.  
4. **Fill** F1..F28 for both `_validation` and `_evaluation` rows with the same `naive_mean`.  
5. **Combine** them into one final `submission.csv` containing 60,980 rows (30,490 `_validation` + 30,490 `_evaluation`).

In [ ]:
import pandas as pd

# Step 1: Load the sales_train_evaluation dataset
sales_train_eval = pd.read_csv('/kaggle/input/sales-train-eval/sales_train_evaluation.csv')

# Step 2: Calculate the average sales for the last 28 days (d_1914 to d_1941)
# Identify the columns for the last 28 days (d_1914 ~ d_1941)
date_columns = [f"d_{i}" for i in range(1914, 1942)]
sales_train_eval["naive_mean"] = sales_train_eval[date_columns].mean(axis=1)

# Step 3: Replicate rows as _validation and adjust the `id` column
# Create a new validation dataframe by replacing '_evaluation' with '_validation' in the id
sales_train_valid = sales_train_eval.copy()
sales_train_valid['id'] = sales_train_valid['id'].str.replace('_evaluation', '_validation')

# Step 4: Fill F1..F28 columns with the naive mean for both _validation and _evaluation
for i in range(1, 29):
    sales_train_eval[f"F{i}"] = sales_train_eval["naive_mean"]
    sales_train_valid[f"F{i}"] = sales_train_valid["naive_mean"]

# Step 5: Combine the _evaluation and _validation data into a single DataFrame
final_submission = pd.concat([sales_train_eval, sales_train_valid], axis=0)

# Step 6: Save the final submission to a CSV file
final_submission.to_csv('/kaggle/working/final_submission.csv', index=False)
print("Final submission saved!")

## Section 5: Improving Naive Submission (LightGBM)
Now, we will load the merged data for a specific store (CA_1) to explore more advanced methods such as feature engineering. This step helps us understand the available columns and prepare for a better predictive model.

In [ ]:
# Load merged data for store CA_1 
df_tx1 = pd.read_pickle("/kaggle/working/merged_TX_1.pkl")

print("Shape of df_tx1:", df_tx1.shape)
print(df_tx1.head(3))

# Check the columns to see what information is available
print("\nColumns in df_tx1:", df_tx1.columns.tolist())

## Creating a Numeric Day Index and Sampling

Now that we have loaded the merged data for TX_1, we can convert the "d" column (e.g. "d_1", "d_2") into a numeric day index. This helps us handle time-based operations more easily. We'll also take a small sample to explore the data without overwhelming memory or visuals.

In [ ]:
# Convert "d" (like "d_1") into an integer day index, and sample a portion
df_tx1["d_num"] = df_tx1["d"].str[2:].astype(int)

print("Added 'd_num' column. First few values:")
print(df_tx1[["d", "d_num"]].head(5))

# Optional: Take a small sample for further exploration
df_tx1_sample = df_tx1.sample(100_000, random_state=42)

print("\nSampled 100,000 rows. Shape:", df_tx1_sample.shape)
print(df_tx1_sample.head(3))

## Analysis of Sample of TX_1

We want to see average sales across the days. This will help get an initial sense of time-based trends without being overwhelmed with the huge dataset.

In [ ]:
# Group by the "date" column to see average sales per day in the sample
avg_sales_by_date = (
    df_tx1_sample
    .groupby("date", as_index=False)["sales"]
    .mean()
    .rename(columns={"sales": "avg_sales_sample"})
)

print("Shape of avg_sales_by_date:", avg_sales_by_date.shape)
print(avg_sales_by_date.head(5))

# (Optional) Sort by date if needed for a clearer chronological view
avg_sales_by_date = avg_sales_by_date.sort_values("date")
print("\nFirst 5 rows after sorting by date:")
print(avg_sales_by_date.head(5))

## Visualizing Sales over Time of Sample TX_1

In [ ]:
# Plot avg_sales_sample vs. date
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(10,4))
sns.lineplot(data=avg_sales_by_date, x="date", y="avg_sales_sample")
plt.title("Daily Average Sales (Sample of 100k rows)")
plt.xlabel("Date")
plt.ylabel("Avg Sales in Sample")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## Extending Feature Engineering

In [ ]:
import pandas as pd
import numpy as np
import os
from lightgbm import LGBMRegressor
from sklearn.model_selection import ParameterGrid
import gc

# List of all store identifiers for processing
all_stores = ["CA_1", "CA_2", "CA_3", "CA_4",
              "TX_1", "TX_2", "TX_3",
              "WI_1", "WI_2", "WI_3"]

# Parameter grid for LightGBM hyperparameter optimization
param_grid = {
    "num_leaves": [31, 63],        # Number of leaves in each tree
    "learning_rate": [0.05, 0.1],  # Learning rate for the model
    "n_estimators": [100, 200]     # Number of trees to fit
}

# Mapping for weekdays to numeric values
weekday_map = {
    "Monday": 1, "Tuesday": 2, "Wednesday": 3,
    "Thursday": 4, "Friday": 5, "Saturday": 6, "Sunday": 7
}

# Dictionary to store metrics for each store
store_metrics = {}

# Iterate over each store to process data and train model
for store in all_stores:
    pickle_path = f"/kaggle/working/merged_{store}.pkl"  # Path to the data file
    
    # Check if the data file exists; if not, skip to the next store
    if not os.path.exists(pickle_path):
        print(f"File not found for {store}, skipping.")
        continue

    print(f"\nProcessing {store} ...")
    
    # Load the merged data for the current store
    df_merged = pd.read_pickle(pickle_path).copy()

    # Convert specified columns to categorical data type to save memory
    for col in ["weekday", "month", "year"]:
        df_merged[col] = df_merged[col].astype("category")
    
    # Convert 'd' column to a numeric format by extracting values
    df_merged["d_num"] = df_merged["d"].str[2:].astype(int)
    df_merged = df_merged.sort_values(["id", "d_num"])  # Sort data by id and date

    # Convert weekday strings to numeric values using the weekday_map
    if df_merged["weekday"].dtype in [object, "category"]:
        df_merged["weekday_num"] = (
            df_merged["weekday"].astype(str)
            .map(weekday_map)
            .fillna(0)
            .astype(int)
        )
    else:
        # If already numeric, fill missing values with 0
        df_merged["weekday_num"] = df_merged["weekday"].fillna(0).astype(int)

    # Convert month and year to integers, ensuring no categorical data type
    df_merged["month"] = df_merged["month"].astype(int).fillna(0).astype(int)
    df_merged["year"]  = df_merged["year"].astype(int).fillna(0).astype(int)

    # Fill missing sell_price values with 0
    df_merged["sell_price"] = df_merged["sell_price"].fillna(0)

    # Create lagged and rolling features for sales
    lags = [7, 14, 28]          # List of lags for sales
    rolling_windows = [7, 28]   # Rolling window sizes for sales
    grouped = df_merged.groupby("id", observed=False)  # Group data by store id
    
    # Generate lagged sales features
    for lag in lags:
        col_name = f"sales_lag{lag}"
        df_merged[col_name] = grouped["sales"].shift(lag)
    
    # Generate rolling mean features for sales
    for w in rolling_windows:
        col_name = f"sales_rollmean{w}"
        df_merged[col_name] = (
            grouped["sales"].shift(1).rolling(w, min_periods=1).mean()
        )
    
    # Fill missing values for snap features
    df_merged[["snap_CA","snap_TX","snap_WI"]] = df_merged[["snap_CA","snap_TX","snap_WI"]].fillna(0)
    
    # Example of additional rolling price feature
    df_merged["price_roll7"] = grouped["sell_price"].shift(1).rolling(7, min_periods=1).mean()
    
    # Prepare a list of feature columns for training
    feature_cols = []
    for lag in lags:
        feature_cols.append(f"sales_lag{lag}")
    for w in rolling_windows:
        feature_cols.append(f"sales_rollmean{w}")
    feature_cols.append("price_roll7")

    # Include additional features needed for the model
    extra_cols = ["snap_CA", "snap_TX", "snap_WI", "sell_price", "weekday_num", "month", "year"]
    feature_cols += extra_cols

    # Fill any remaining missing values in feature columns with 0
    df_merged[feature_cols] = df_merged[feature_cols].fillna(0)

    # Split data into training and validation sets based on the time (d_num value)
    df_train = df_merged[df_merged["d_num"] < 1800].copy()  # Training data
    df_val   = df_merged[df_merged["d_num"] >= 1800].copy()  # Validation data

    # Check if there's enough data for training and validation
    if len(df_train) == 0 or len(df_val) == 0:
        print(f"No data for {store}, skipping training.")
        continue

    # Define training and validation features and labels
    X_train = df_train[feature_cols]
    y_train = df_train["sales"]
    X_val   = df_val[feature_cols]
    y_val   = df_val["sales"]

    best_rmse = float("inf")  # Initialize best RMSE
    best_params = None         # Initialize variable to store best parameters

    # Train the model using GridSearch over the parameter grid
    for params in ParameterGrid(param_grid):
        model = LGBMRegressor(random_state=42, **params)  # Initialize model with parameters
        model.fit(X_train, y_train)  # Fit model to training data

        # Predict validation data
        pred_val = model.predict(X_val)
        rmse = np.sqrt(np.mean((pred_val - y_val)**2))  # Calculate RMSE for validation predictions

        # Update best RMSE and parameters if current RMSE is lower
        if rmse < best_rmse:
            best_rmse = rmse
            best_params = params

    # Store the best parameters and RMSE for the current store
    print(f"{store} best params: {best_params}, RMSE: {best_rmse:.4f}")
    store_metrics[store] = (best_params, best_rmse)

    # Clear memory for the current store's data to optimize memory usage
    del df_merged, df_train, df_val, X_train, X_val, y_train, y_val
    gc.collect()  # Force garbage collection

# Print all store metrics at the end of processing
print("\nAll store metrics:")
for st, (pars, sc) in store_metrics.items():
    print(f"{st} -> RMSE: {sc:.4f}, best params: {pars}")

In [ ]:
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input, Dropout
import tensorflow.keras.backend as K
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
import gc
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint

# Enable multi-GPU strategy for TensorFlow to improve performance during training
strategy = tf.distribute.MirroredStrategy()

# Enable memory growth on available GPU devices to prevent allocation errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

# List of all store identifiers to process
all_stores = ["CA_1", "CA_2", "CA_3", "CA_4",
              "TX_1", "TX_2", "TX_3",
              "WI_1", "WI_2", "WI_3"]

# Create a CSV file to store LightGBM predictions
lgbm_file = open("submission_lgbm.csv", "w")
lgbm_file.write("id," + ",".join([f"F{i}" for i in range(1, 29)]) + "\n")

# Process each store's data
for store in all_stores:
    # Load the pre-processed data for the current store
    pickle_path = f"/kaggle/working/merged_{store}.pkl"
    if not os.path.exists(pickle_path):
        print(f"No file for {store}, skipping.")
        continue  # Skip if the data file does not exist

    print(f"\nProcessing {store}...")
    # Read the data into a DataFrame
    df_merged = pd.read_pickle(pickle_path)
    df_merged["d_num"] = df_merged["d"].str[2:].astype(int)  # Convert date to numerical format
    df_merged = df_merged.sort_values(["id", "d_num"])  # Sort data by 'id' and 'd_num'
    
    # Load the best hyperparameters for the LightGBM model for this store
    best_params = store_metrics[store][0]
    
    # Define feature columns by creating lagged and rolling features
    lags = [7, 14, 28]  # List of lag periods for sales
    rolling_windows = [7, 28]  # List of rolling windows for sales averages
    
    # Generate lagged sales features
    for lag in lags:
        df_merged[f"sales_lag{lag}"] = df_merged["sales"].shift(lag)

    # Generate rolling mean features for sales over specified windows
    for window in rolling_windows:
        df_merged[f"sales_rollmean{window}"] = df_merged["sales"].rolling(window=window).mean()

    # Compute additional price-related features
    df_merged["price_roll7"] = df_merged["sell_price"].rolling(window=7).mean()

    # Fill missing values in feature columns with 0 and convert to float32 type
    feature_cols = [f"sales_lag{x}" for x in lags] + [f"sales_rollmean{x}" for x in rolling_windows]
    feature_cols += ["sell_price", "price_roll7"]
    df_merged[feature_cols] = df_merged[feature_cols].fillna(0).astype(np.float32)

    # Prepare features and labels for training LightGBM
    X_lgbm = df_merged[feature_cols].values  # Feature matrix for LightGBM
    y_lgbm = df_merged["sales"].values       # Target variable (sales)

    # Train the LightGBM model using the specified hyperparameters
    lgbm_model = LGBMRegressor(random_state=42, **best_params, device="gpu", gpu_platform_id=0, gpu_device_id=0)
    # Create a validation set by splitting data
    X_train_lgbm, X_val_lgbm, y_train_lgbm, y_val_lgbm = train_test_split(X_lgbm, y_lgbm, test_size=0.1, random_state=42)

    # Fit the model while tracking validation performance
    lgbm_model.fit(X_train_lgbm, y_train_lgbm,
                   eval_set=[(X_train_lgbm, y_train_lgbm), (X_val_lgbm, y_val_lgbm)],
                   eval_metric="rmse")

    # Extract RMSE results from training
    results = lgbm_model.evals_result_
    train_rmse = results['training']['rmse']
    valid_rmse = results['valid_1']['rmse']

    # Plot RMSE curves for training and validation
    plt.plot(train_rmse, label='Train RMSE')
    plt.plot(valid_rmse, label='Validation RMSE')
    plt.xlabel('Iteration')
    plt.ylabel('RMSE')
    plt.title(f'Training and Validation RMSE curves - Store {store}')
    plt.legend()
    plt.savefig(f"LGBM_Training_Validation_Curve_{store}.png", dpi=300)  # Save plot as a PNG file
    plt.show()  # Display plot

    # Predict future sales using the trained LightGBM model
    future_days = np.arange(1942, 1970)  # Define future days for prediction
    all_ids = df_merged["id"].unique()  # Get unique store IDs
    # Create a DataFrame for future predictions
    future_df = pd.DataFrame({"id": np.repeat(all_ids, len(future_days)), 
                              "d_num": np.tile(future_days, len(all_ids)),
                              "sales": np.nan})
    
    # Concatenate future data with the original dataset
    df_merged = pd.concat([df_merged, future_df], ignore_index=True, sort=False)
    df_merged = df_merged.sort_values(["id", "d_num"]).reset_index(drop=True)

    # Prepare feature data for future predictions
    future_features = df_merged[feature_cols].iloc[-len(future_df):].values
    
    # Generate predictions for future sales
    predictions_lgbm = lgbm_model.predict(future_features)
    
    # Save the predictions to the CSV file
    for i, pred in enumerate(predictions_lgbm):
        lgbm_file.write(f"{future_df['id'].iloc[i]},{','.join([str(pred) for _ in range(28)])}\n")

    # Clean up memory by deleting temporary variables
    del df_merged, X_lgbm, y_lgbm, predictions_lgbm, future_features
    gc.collect()  # Force garbage collection

# Close the LightGBM results CSV file after writing all predictions
lgbm_file.close()
print("LightGBM results saved to submission_lgbm.csv")  # Confirmation of saved results

In [ ]:
!pip install keras-tuner

In [ ]:
import matplotlib.pyplot as plt
import keras_tuner as kt
import numpy as np
import pandas as pd
import gc
import os
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# RMSSE custom metric definition
def rmsse_wrapper(training_data):
    """
    Wrapper function to compute Root Mean Squared Scaled Error (RMSSE) as a custom metric.
    The metric is based on the true values and model predictions normalized by the training data.
    """
    training_data = tf.convert_to_tensor(training_data, dtype=tf.float32)  # Convert training data to tensor for calculations
    training_mean = tf.reduce_mean(training_data)  # Compute mean of the training data

    def rmsse(y_true, y_pred):
        """
        Computes RMSSE by normalizing RMSE with the mean squared differences in training data.
        Args:
            y_true: The true sales values.
            y_pred: The predicted sales values by the model.

        Returns:
            The scaled error as a tensor.
        """
        # Calculate the scaled root mean squared error
        scaled_error = tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred))) / tf.sqrt(tf.reduce_mean(tf.square(training_data - training_mean)))
        return scaled_error

    return rmsse  # Return the RMSSE computation function


# Function to create LSTM-compatible data
def create_lstm_data(df, feature_cols, seq_length=28):
    """
    Prepares input sequences for LSTM model training.
    Args:
        df: The DataFrame containing the data.
        feature_cols: List of feature columns to use for the model.
        seq_length: The number of time steps to include in each input sequence.

    Returns:
        Tuple containing:
        - Prepared input sequences for LSTM
        - Scaled outputs (sales values)
        - Scaler for features
        - Scaler for targets (sales)
    """
    scaler_x = StandardScaler()  # Scaler for feature normalization
    scaler_y = StandardScaler()  # Scaler for target normalization
    scaled_features = scaler_x.fit_transform(df[feature_cols].astype(np.float32))  # Scale features to standard Gaussian

    X, y = [], []
    for i in range(seq_length, len(df)):
        # Create sequences of features for LSTM input
        X.append(scaled_features[i-seq_length:i])
        y.append(df["sales"].values[i])  # Append corresponding output (sales) for each sequence

    y = np.array(y, dtype=np.float32).reshape(-1, 1)  # Reshape target values for scaling
    y_scaled = scaler_y.fit_transform(y).flatten()  # Scale target values

    return np.array(X, dtype=np.float32), y_scaled, scaler_x, scaler_y  # Return feature and target data


def build_lstm_model_with_hp(hp, input_shape, training_data):
    """
    Builds an LSTM model with hyperparameter tuning using Keras Tuner.
    Args:
        hp: Hyperparameter object for tuning.
        input_shape: Shape of the input data.
        training_data: The training data used for RMSSE calculation.

    Returns:
        Compiled LSTM model with specified hyperparameters.
    """
    model = Sequential()  # Initialize sequential model

    # Define the number of LSTM units with hyperparameter tuning
    units = hp.Int('units', min_value=32, max_value=128, step=32)

    # First input LSTM layer
    model.add(LSTM(units, return_sequences=True, input_shape=input_shape))

    # Hyperparameter search for additional LSTM layers
    num_lstm_layers = hp.Int('num_lstm_layers', min_value=1, max_value=3, step=1)
    
    for _ in range(num_lstm_layers - 1):  # Add additional LSTM layers if needed
        model.add(LSTM(units, return_sequences=True))

    # Final LSTM layer
    model.add(LSTM(units, return_sequences=False))

    # Fully connected Dense layers
    model.add(Dense(hp.Int('dense_units', min_value=32, max_value=128, step=32), activation='relu'))  # Hidden layer
    model.add(Dense(1))  # Output layer for predictions

    # Hyperparameter search for learning rate and optimizer
    learning_rate = hp.Float('learning_rate', min_value=1e-5, max_value=1e-2, sampling='LOG')
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)  # Adam optimizer with variable learning rate

    # Compile the model with mean squared error loss and RMSSE as the metric
    model.compile(loss='mse', optimizer=optimizer, metrics=[rmsse_wrapper(training_data)])
    
    return model  # Return the constructed and compiled model
    

def tune_lstm_model(X_train_lstm, y_train_lstm, input_shape, training_data):
    """
    Uses Keras Tuner to search for the best hyperparameters for the LSTM model.
    Args:
        X_train_lstm: Training feature data for LSTM.
        y_train_lstm: Training target data (sales).
        input_shape: Shape of the input data.
        training_data: The raw training data used for RMSSE normalization.

    Returns:
        The best model found during hyperparameter tuning.
    """
    tuner = kt.Hyperband(
        lambda hp: build_lstm_model_with_hp(hp, input_shape, training_data),  # Tuner to build models with specific hyperparameters
        objective='val_loss',  # Objective metric to minimize
        max_epochs=20,  # Maximum number of epochs for training
        factor=3,  # Proportion of trials to keep for the next round
        directory='lstm_tuning',  # Directory to save tuning results
        project_name='lstm_search',  # Project name for organizing tunings
        overwrite=True  # Overwrite existing results
    )

    # Run hyperparameter tuning on the training data
    tuner.search(X_train_lstm, y_train_lstm, epochs=20, validation_split=0.1)
    best_hps = tuner.get_best_hyperparameters(num_trials=5)[0]  # Get the best hyperparameters
    print("Best Hyperparameters:", best_hps.values)  # Log the best hyperparameters

    # Build and return the model using the best hyperparameters
    best_model = build_lstm_model_with_hp(best_hps, input_shape, training_data)
    
    return best_model


# Open file to store predictions for later submission
lstm_file = open("submission_lstm.csv", "w")
lstm_file.write("id," + ",".join([f"F{i}" for i in range(1, 29)]) + "\n")  # Write CSV header with forecast indicators

# Process each store's data in the list of stores
for store in all_stores:
    pickle_path = f"/kaggle/working/merged_{store}.pkl"  # Path to store-specific data file
    if not os.path.exists(pickle_path):  # Check if the data file exists
        print(f"No file for {store}, skipping.")
        continue  # Skip if not found

    print(f"\nProcessing {store}...")
    df_merged = pd.read_pickle(pickle_path)  # Load data into DataFrame
    df_merged["d_num"] = df_merged["d"].str[2:].astype(int)  # Convert date column to numeric format
    df_merged = df_merged.sort_values(["id", "d_num"])  # Sort values by store ID and date
    
    # Feature engineering: Create lagged sales and rolling mean features
    lags = [7, 14, 28]  # Lag periods for previous sales
    rolling_windows = [7, 28]  # Rolling mean computation windows
    feature_cols = [f"sales_lag{x}" for x in lags] + [f"sales_rollmean{x}" for x in rolling_windows]
    feature_cols += ["sell_price", "price_roll7"]
    
    # Create lagged and rolling features
    for lag in lags:
        df_merged[f"sales_lag{lag}"] = df_merged["sales"].shift(lag)
    for window in rolling_windows:
        df_merged[f"sales_rollmean{window}"] = df_merged["sales"].rolling(window=window).mean()
    df_merged["price_roll7"] = df_merged["sell_price"].rolling(window=7).mean()
    
    # Handle missing values in feature columns by filling with column mean
    df_merged[feature_cols] = df_merged[feature_cols].fillna(df_merged[feature_cols].mean())

    # Prepare and scale LSTM training data
    X_train_lstm, y_train_lstm, scaler_x, scaler_y = create_lstm_data(df_merged, feature_cols)
    
    # Tune LSTM model to find best hyperparameters
    best_lstm_model = tune_lstm_model(X_train_lstm, y_train_lstm, (X_train_lstm.shape[1], X_train_lstm.shape[2]), df_merged["sales"].values)
    
    # Train the best model found during hyperparameter tuning
    history = best_lstm_model.fit(X_train_lstm, y_train_lstm, epochs=20, verbose=1, validation_split=0.1)
    
    # Plot and visualize training and validation RMSSE for each epoch
    plt.figure(figsize=(12, 6))
    plt.plot(history.history['rmsse'], label='Training RMSSE')
    plt.plot(history.history['val_rmsse'], label='Validation RMSSE')
    plt.title(f"Training and Validation RMSSE for {store}")  # Title for the plot
    plt.xlabel('Epochs')  # X-axis label
    plt.ylabel('RMSSE')  # Y-axis label
    plt.legend()  # Show legend
    plt.grid(True)  # Add grid
    plt.savefig(f'RMSSE_curve_{store}.png')  # Save plot as an image
    plt.close()  # Close the plot to free memory
    
    # Prepare for future sales predictions using the LSTM model
    future_lstm_data = np.array([X_train_lstm[-1]])  # Start predictions from the last known input sequence
    predictions_lstm = []  # List to store future predictions
    for t in range(28):  # Generate predictions for the next 28 time steps
        pred = best_lstm_model.predict(future_lstm_data)[0][0]  # Predict the next sales value
        predictions_lstm.append(pred)  # Store the prediction
        # Roll the future data for the next prediction
        future_lstm_data = np.roll(future_lstm_data, shift=-1, axis=1)
        future_lstm_data[0, -1, 0] = pred  # Update the last feature with the predicted sales value
    
    # Inverse scale the predictions to retrieve original sales values
    predictions_lstm = scaler_y.inverse_transform(np.array(predictions_lstm).reshape(-1, 1)).flatten()
    
    # Save predictions to the CSV file for submission
    for i, pred in enumerate(predictions_lstm):
        lstm_file.write(f"{df_merged['id'].iloc[i]},{','.join([str(pred) for _ in range(28)])}\n")  # Write predictions for future time steps

    # Clear memory of the DataFrame and arrays after processing each store
    del df_merged, X_train_lstm, y_train_lstm, future_lstm_data, predictions_lstm
    gc.collect()  # Perform garbage collection to free memory

lstm_file.close()  # Close the predictions file
print("LSTM results saved to submission_lstm.csv")  # Confirmation message after saving results